# Add Memory to OpenAI Functions Agent
---

Este cuaderno explica cómo agregar memoria a un agente de OpenAI Functions

Agregemos las dependencias basicas para este ejemplo:

In [4]:
from dotenv import load_dotenv, find_dotenv
from langchain.chains import LLMMathChain
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper, WikipediaAPIWrapper
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

load_dotenv(find_dotenv())

True

In [7]:
# llm
llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)

Definamos la tool para busquedas en internet, usando `SerpAPIWrapper`. asi como una tool para hacer calculos matematicos usando `LLMMathChain`

**Nota:** A diferencia del ejemplo de python.langchain.com, se elimina el codigo de SQLDatabaseChain

In [9]:
search = SerpAPIWrapper()
wikipedia = WikipediaAPIWrapper()
math = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    Tool(
        name="Calculator",
        func=math.run,
        description="useful for when you need to answer questions about math",
    ),
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="useful for when you need answer with wikipedia info",
    )
]

Agregaremos la memoria para el agente

In [10]:
agent_kwargs = {
    "extra_prompt_messages": [
        MessagesPlaceholder(variable_name="memory")
    ]
}

memory = ConversationBufferMemory(
    memory_key="memory",
    return_messages=True,
)

Inicializamos el agente

In [11]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    agent_kwargs=agent_kwargs,
    memory=memory,
    verbose=True,
)

Enviemos un mensaje

In [12]:
agent.run("hi!")



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


'Hello! How can I assist you today?'

In [13]:
agent.run("my name is bob")



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


'Hello Bob! How can I assist you today?'

In [14]:
agent.run("whats my name")



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.


'Your name is Bob.'